# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
/Users/susannahan/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/susannahan/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/susannahan/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 177us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 80us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 79us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 81us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 70us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 82us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 85us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 68us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 68us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 64us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 69us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 62us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 61us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 96us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 102us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 90us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 82us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 94us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 91us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 8

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model 
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 200us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 91us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 81us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 70us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 73us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 73us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 80us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 74us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 61us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 70us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 95us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 92us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 87us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 81us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 69us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 75us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 80us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 87us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 101us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 6

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [12]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 219us/step - loss: 0.4368 - mean_squared_error: 0.4368 - val_loss: 0.1991 - val_mean_squared_error: 0.1991
Epoch 2/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.2275 - mean_squared_error: 0.2275 - val_loss: 0.1613 - val_mean_squared_error: 0.1613
Epoch 3/150
1051/1051 [==============================] - 0s 96us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 0.1587 - val_mean_squared_error: 0.1587
Epoch 4/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.1642 - mean_squared_error: 0.1642 - val_loss: 0.1376 - val_mean_squared_error: 0.1376
Epoch 5/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.1481 - mean_squared_error: 0.1481 - val_loss: 0.1320 - val_mean_squared_error: 0.1320
Epoch 6/150
1051/1051 [==============================] - 0s 101us/step - loss: 0.1329 - mean_squared_error: 0.1329 - val_l

Epoch 50/150
1051/1051 [==============================] - 0s 97us/step - loss: 0.0318 - mean_squared_error: 0.0318 - val_loss: 0.1014 - val_mean_squared_error: 0.1014
Epoch 51/150
1051/1051 [==============================] - 0s 102us/step - loss: 0.0317 - mean_squared_error: 0.0317 - val_loss: 0.1029 - val_mean_squared_error: 0.1029
Epoch 52/150
1051/1051 [==============================] - 0s 83us/step - loss: 0.0309 - mean_squared_error: 0.0309 - val_loss: 0.1031 - val_mean_squared_error: 0.1031
Epoch 53/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0302 - mean_squared_error: 0.0302 - val_loss: 0.1030 - val_mean_squared_error: 0.1030
Epoch 54/150
1051/1051 [==============================] - 0s 90us/step - loss: 0.0292 - mean_squared_error: 0.0292 - val_loss: 0.1045 - val_mean_squared_error: 0.1045
Epoch 55/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0286 - mean_squared_error: 0.0286 - val_loss: 0.1037 - val_mean_squared_error: 0.10

1051/1051 [==============================] - 0s 89us/step - loss: 0.0143 - mean_squared_error: 0.0143 - val_loss: 0.1116 - val_mean_squared_error: 0.1116
Epoch 100/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1107 - val_mean_squared_error: 0.1107
Epoch 101/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1122 - val_mean_squared_error: 0.1122
Epoch 102/150
1051/1051 [==============================] - 0s 84us/step - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.1115 - val_mean_squared_error: 0.1115
Epoch 103/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1121 - val_mean_squared_error: 0.1121
Epoch 104/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0133 - mean_squared_error: 0.0133 - val_loss: 0.1126 - val_mean_squared_error: 0.1126
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1168 - val_mean_squared_error: 0.1168
Epoch 149/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1165 - val_mean_squared_error: 0.1165
Epoch 150/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1177 - val_mean_squared_error: 0.1177


Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [13]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 37us/step


[0.007792733689428056, 0.007792733689428056]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [14]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 69us/step


[0.11768812779691283, 0.11768812779691283]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [15]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

26958.6308264894

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [16]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))


# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 215us/step - loss: 0.5084 - mean_squared_error: 0.5084 - val_loss: 0.1704 - val_mean_squared_error: 0.1704
Epoch 2/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.2216 - mean_squared_error: 0.2216 - val_loss: 0.1430 - val_mean_squared_error: 0.1430
Epoch 3/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.1796 - mean_squared_error: 0.1796 - val_loss: 0.1381 - val_mean_squared_error: 0.1381
Epoch 4/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.1565 - mean_squared_error: 0.1565 - val_loss: 0.1249 - val_mean_squared_error: 0.1249
Epoch 5/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.1405 - mean_squared_error: 0.1405 - val_loss: 0.1192 - val_mean_squared_error: 0.1192
Epoch 6/150
1051/1051 [==============================] - 0s 86us/step - loss: 0.1265 - mean_squared_error: 0.1265 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 102us/step - loss: 0.0305 - mean_squared_error: 0.0305 - val_loss: 0.1088 - val_mean_squared_error: 0.1088
Epoch 51/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0298 - mean_squared_error: 0.0298 - val_loss: 0.1086 - val_mean_squared_error: 0.1086
Epoch 52/150
1051/1051 [==============================] - 0s 103us/step - loss: 0.0294 - mean_squared_error: 0.0294 - val_loss: 0.1096 - val_mean_squared_error: 0.1096
Epoch 53/150
1051/1051 [==============================] - 0s 105us/step - loss: 0.0289 - mean_squared_error: 0.0289 - val_loss: 0.1098 - val_mean_squared_error: 0.1098
Epoch 54/150
1051/1051 [==============================] - 0s 105us/step - loss: 0.0283 - mean_squared_error: 0.0283 - val_loss: 0.1100 - val_mean_squared_error: 0.1100
Epoch 55/150
1051/1051 [==============================] - 0s 99us/step - loss: 0.0279 - mean_squared_error: 0.0279 - val_loss: 0.1095 - val_mean_squared_error: 0

Epoch 99/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1186 - val_mean_squared_error: 0.1186
Epoch 100/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1186 - val_mean_squared_error: 0.1186
Epoch 101/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1186 - val_mean_squared_error: 0.1186
Epoch 102/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0139 - mean_squared_error: 0.0139 - val_loss: 0.1177 - val_mean_squared_error: 0.1177
Epoch 103/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1178 - val_mean_squared_error: 0.1178
Epoch 104/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0135 - mean_squared_error: 0.0135 - val_loss: 0.1204 - val_mean_squared_error: 

Epoch 148/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1236 - val_mean_squared_error: 0.1236
Epoch 149/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1232 - val_mean_squared_error: 0.1232
Epoch 150/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0083 - mean_squared_error: 0.0083 - val_loss: 0.1235 - val_mean_squared_error: 0.1235


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 73us/step


[0.00812792454666319, 0.00812792454666319]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 69us/step


[0.12352494551258396, 0.12352494551258396]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [19]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 252us/step - loss: 0.4768 - mean_squared_error: 0.4768 - val_loss: 0.1460 - val_mean_squared_error: 0.1460
Epoch 2/150
1051/1051 [==============================] - 0s 102us/step - loss: 0.2095 - mean_squared_error: 0.2095 - val_loss: 0.1230 - val_mean_squared_error: 0.1230
Epoch 3/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.1698 - mean_squared_error: 0.1698 - val_loss: 0.1168 - val_mean_squared_error: 0.1168
Epoch 4/150
1051/1051 [==============================] - 0s 91us/step - loss: 0.1479 - mean_squared_error: 0.1479 - val_loss: 0.1083 - val_mean_squared_error: 0.1083
Epoch 5/150
1051/1051 [==============================] - 0s 84us/step - loss: 0.1330 - mean_squared_error: 0.1330 - val_loss: 0.1058 - val_mean_squared_error: 0.1058
Epoch 6/150
1051/1051 [==============================] - 0s 113us/step - loss: 0.1197 - mean_squared_error: 0.1197 - val_

1051/1051 [==============================] - 0s 94us/step - loss: 0.0276 - mean_squared_error: 0.0276 - val_loss: 0.1191 - val_mean_squared_error: 0.1191
Epoch 50/150
1051/1051 [==============================] - 0s 84us/step - loss: 0.0266 - mean_squared_error: 0.0266 - val_loss: 0.1160 - val_mean_squared_error: 0.1160
Epoch 51/150
1051/1051 [==============================] - 0s 90us/step - loss: 0.0261 - mean_squared_error: 0.0261 - val_loss: 0.1172 - val_mean_squared_error: 0.1172
Epoch 52/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.0258 - mean_squared_error: 0.0258 - val_loss: 0.1186 - val_mean_squared_error: 0.1186
Epoch 53/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.0252 - mean_squared_error: 0.0252 - val_loss: 0.1191 - val_mean_squared_error: 0.1191
Epoch 54/150
1051/1051 [==============================] - 0s 94us/step - loss: 0.0246 - mean_squared_error: 0.0246 - val_loss: 0.1189 - val_mean_squared_error: 0.1189
Epoch 55/15

1051/1051 [==============================] - 0s 74us/step - loss: 0.0129 - mean_squared_error: 0.0129 - val_loss: 0.1268 - val_mean_squared_error: 0.1268
Epoch 99/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0126 - mean_squared_error: 0.0126 - val_loss: 0.1294 - val_mean_squared_error: 0.1294
Epoch 100/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1286 - val_mean_squared_error: 0.1286
Epoch 101/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1293 - val_mean_squared_error: 0.1293
Epoch 102/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0123 - mean_squared_error: 0.0123 - val_loss: 0.1273 - val_mean_squared_error: 0.1273
Epoch 103/150
1051/1051 [==============================] - 0s 74us/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.1282 - val_mean_squared_error: 0.1282
Epoch 1

Epoch 147/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0074 - mean_squared_error: 0.0074 - val_loss: 0.1342 - val_mean_squared_error: 0.1342
Epoch 148/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1353 - val_mean_squared_error: 0.1353
Epoch 149/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1336 - val_mean_squared_error: 0.1336
Epoch 150/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1348 - val_mean_squared_error: 0.1348


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [ ]:
# Evaluate the model on training data


Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [ ]:
# Evaluate the model on validate data


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [ ]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model


# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 32us/step


[0.006983263652016729, 0.006983263652016729]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 91us/step


[0.13478767711088457, 0.13478767711088457]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [23]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 348us/step - loss: 0.3806 - mean_squared_error: 0.3806 - val_loss: 0.1655 - val_mean_squared_error: 0.1655
Epoch 2/150
1051/1051 [==============================] - 0s 94us/step - loss: 0.1611 - mean_squared_error: 0.1611 - val_loss: 0.1333 - val_mean_squared_error: 0.1333
Epoch 3/150
1051/1051 [==============================] - 0s 78us/step - loss: 0.1128 - mean_squared_error: 0.1128 - val_loss: 0.1042 - val_mean_squared_error: 0.1042
Epoch 4/150
1051/1051 [==============================] - 0s 99us/step - loss: 0.0862 - mean_squared_error: 0.0862 - val_loss: 0.1015 - val_mean_squared_error: 0.1015
Epoch 5/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.0740 - mean_squared_error: 0.0740 - val_loss: 0.1016 - val_mean_squared_error: 0.1016
Epoch 6/150
1051/1051 [==============================] - 0s 101us/step - loss: 0.0554 - mean_squared_error: 0.0554 - val_l

Epoch 50/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0060 - mean_squared_error: 0.0060 - val_loss: 0.0999 - val_mean_squared_error: 0.0999
Epoch 51/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0049 - mean_squared_error: 0.0049 - val_loss: 0.1067 - val_mean_squared_error: 0.1067
Epoch 52/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0948 - val_mean_squared_error: 0.0948
Epoch 53/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.1018 - val_mean_squared_error: 0.1018
Epoch 54/150
1051/1051 [==============================] - 0s 74us/step - loss: 0.0025 - mean_squared_error: 0.0025 - val_loss: 0.0953 - val_mean_squared_error: 0.0953
Epoch 55/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.1038 - val_mean_squared_error: 0.103

1051/1051 [==============================] - 0s 99us/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0959 - val_mean_squared_error: 0.0959
Epoch 100/150
1051/1051 [==============================] - 0s 74us/step - loss: 0.0049 - mean_squared_error: 0.0049 - val_loss: 0.1031 - val_mean_squared_error: 0.1031
Epoch 101/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0950 - val_mean_squared_error: 0.0950
Epoch 102/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.1009 - val_mean_squared_error: 0.1009
Epoch 103/150
1051/1051 [==============================] - 0s 83us/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0909 - val_mean_squared_error: 0.0909
Epoch 104/150
1051/1051 [==============================] - 0s 91us/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0977 - val_mean_squared_error: 0.0977
Epoch 

1051/1051 [==============================] - 0s 80us/step - loss: 0.0050 - mean_squared_error: 0.0050 - val_loss: 0.0947 - val_mean_squared_error: 0.0947
Epoch 148/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.0862 - val_mean_squared_error: 0.0862
Epoch 149/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0955 - val_mean_squared_error: 0.0955
Epoch 150/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0907 - val_mean_squared_error: 0.0907


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 73us/step


[0.00303471846781586, 0.00303471846781586]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 52us/step


[0.09066739528238094, 0.09066739528238094]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [26]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 118us/step


[0.13929647125609934, 0.13929647125609934]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [27]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29329.289512543735

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.